# GPU programming using PyOpenCL, part 5: Scan

This is a set of exercises on the usage of PyOpenCL. There are 6 main classes of parallel algorithms:
1. Map or Element-wise kernels: 1 thread calculates 1 result from 1 input position
2. Gather: 1 thread calculates 1 result from several input data, one typical example is the convolution
3. Scatter: 1 thread uses 1 input element and scatters it on one or several output pixels, this requires the usage of atomic operarions
4. Reduction: Apply the same associative operation on all element of an ensemble, for example the sum of all elements in a list.
5. Scan: also called *prefix sum*, this algorithm applies the same associative operation to all *previous* elements of a list, for example a cummulative sum (cumsum)
6. Sort: using sorting network like the bitonic sort.

This fith tutorial focuses on the **Scan** operation where one applies the same associative operation to all element previously in the input array. 
The result is an array with as many elements as the input array!



Two main algorithms exists: 

* [Hillis and Steele](https://en.wikipedia.org/wiki/Prefix_sum?Parallel%20algorithms#Algorithm_1:_Shorter_span,_more_parallel)

 ![Hillis and Steele](Hillis_Steele.svg)

* [Blelloch](https://en.wikipedia.org/wiki/Prefix_sum?Parallel%20algorithms#Algorithm_2:_Work-efficient)

 ![Blelloch](Prefix_sum_16.svg)
  

The later algorithm is similar to the reduction algorythm applies twice. Dues to the limited time, this algorithm will only be demonstrates using metaprogramming.

## Now it is your turn:

Write a byte-offset algorithm that substract the previous value from the current and stores the results on:
* 1 byte if the value is between -127 and 127
* 3 bytes (-128+value on 2 bytes little endian) if the difference value is between -32767 and 32767
* 7 bytes (-128, -128, 0 + value on 4 bytes little endian) if the difference value is larger.


In [9]:
import numpy as np
size = 100
ary = np.random.poisson(100, size=size)

In [10]:
def compByteOffset_numpy(data):
    """
    Compress a dataset into a string using the byte_offet algorithm

    :param data: ndarray
    :return: string/bytes with compressed data

    test = numpy.array([0,1,2,127,0,1,2,128,0,1,2,32767,0,1,2,32768,0,1,2,2147483647,0,1,2,2147483648,0,1,2,128,129,130,32767,32768,128,129,130,32768,2147483647,2147483648])

    """
    flat = numpy.ascontiguousarray(data.ravel(), numpy.int64)
    delta = numpy.zeros_like(flat)
    delta[0] = flat[0]
    delta[1:] = flat[1:] - flat[:-1]
    mask = abs(delta) > 127
    exceptions = numpy.nonzero(mask)[0]
    if numpy.little_endian:
        byteswap = False
    else:
        byteswap = True
    start = 0
    binary_blob = b""
    for stop in exceptions:
        if stop - start > 0:
            binary_blob += delta[start:stop].astype(numpy.int8).tobytes()
        exc = delta[stop]
        absexc = abs(exc)
        if absexc > 2147483647:  # 2**31-1
            binary_blob += b"\x80\x00\x80\x00\x00\x00\x80"
            if byteswap:
                binary_blob += delta[stop:stop + 1].byteswap().tobytes()
            else:
                binary_blob += delta[stop:stop + 1].tobytes()
        elif absexc > 32767:  # 2**15-1
            binary_blob += b"\x80\x00\x80"
            if byteswap:
                binary_blob += delta[stop:stop + 1].astype(numpy.int32).byteswap().tobytes()
            else:
                binary_blob += delta[stop:stop + 1].astype(numpy.int32).tobytes()
        else:  # >127
            binary_blob += b"\x80"
            if byteswap:
                binary_blob += delta[stop:stop + 1].astype(numpy.int16).byteswap().tobytes()
            else:
                binary_blob += delta[stop:stop + 1].astype(numpy.int16).tobytes()
        start = stop + 1
    if start < delta.size:
        binary_blob += delta[start:].astype(numpy.int8).tobytes()
    return binary_blob

In [11]:
ref = compByteOffset_numpy(ary)

In [12]:
ref

b'c\xfb\x05\r\xf7\xfd\t\xf7\xf9\xf0\x1d\xf9\x05\xfd\xff\xf2\x0e\x08\xef\x0c\x05\xfc\xf5\x0f\xfc\r\xe7\x01\x06\x07\xfc\x01\x02\x00\xe5\x15\x0c\xf8\r\xf6\xfe\xff\xfc\x05\xff\xf0\x06\xfe\x1c\xee\x02\xf9\xfd\x0b\xfc\x06\xf4\t\x03\xf1\x13\xf3\xfd&\xd5\x06\xf7\xf5\r\x0c\x03\xfa\x07\xef\x1a\xf0\xf3\x17\xe9\x18\xf0\x14\xfb\xf6\t\xf8\x1c\xd6\x14\xf0\xff\x1d\xec\x01\x0c\x03\xf9\xfd\x02\xef'